In [ ]:
!pip install mediapipe opencv-python
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)
mp_drawing = mp.solutions.drawing_utils

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [ ]:
def draw_text_with_background(image, text, position, text_color, bg_color, font_scale=1.3, thickness=2):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    bottom_left = position
    top_right = (position[0] + text_size[0], position[1] - text_size[1] - 10)
    text_position = (bottom_left[0], bottom_left[1] - 5)
    cv2.rectangle(image, bottom_left, top_right, bg_color, cv2.FILLED)
    cv2.putText(image, text, text_position, font, font_scale, text_color, thickness)

In [ ]:
def draw_angles_with_background(image, angle, text_pos):
    rectangle_color = (128, 128, 128)
    text_color = (255, 255, 255)
    text_size = cv2.getTextSize(str(angle), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
    cv2.rectangle(image, (text_pos[0], text_pos[1] - text_size[1] - 2),
                  (text_pos[0] + text_size[0], text_pos[1]), rectangle_color, -1)
    cv2.putText(image, str(angle), text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                text_color, 2, cv2.LINE_AA)

In [ ]:
# Use the uploaded filename
video_path = '/content/WIN_20250411_19_34_31_Pro.mp4'
cap = cv2.VideoCapture(video_path)
frame_count = 0

In [ ]:
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > 300:  # limit to 300 frames for display
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        image = frame.copy()

        try:
            landmarks = results.pose_landmarks.landmark

            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            left_angle = round(calculate_angle(left_shoulder, left_elbow, left_wrist), 2)
            right_angle = round(calculate_angle(right_shoulder, right_elbow, right_wrist), 2)

            left_shoulder_angle = round(calculate_angle(left_elbow, left_shoulder, right_shoulder), 2)
            right_shoulder_angle = round(calculate_angle(right_elbow, right_shoulder, left_shoulder), 2)

            if left_angle < 60 or right_angle < 60 or left_angle > 110 or right_angle > 110 or \
               left_shoulder_angle < 140 or right_shoulder_angle < 140:
                draw_text_with_background(image, "WRONG POSTURE: CHECK THE ANGLES!", (20, 40), (0, 0, 255), (0, 0, 0))
            else:
                draw_text_with_background(image, "RIGHT POSTURE! GOOD WORK!", (20, 40), (0, 255, 0), (0, 0, 0))

            text_pos_left = tuple(np.add(np.multiply(left_elbow, [image.shape[1], image.shape[0]]).astype(int), (-20, -20)))
            text_pos_right = tuple(np.add(np.multiply(right_elbow, [image.shape[1], image.shape[0]]).astype(int), (-20, -20)))
            text_pos_shoulder_left = tuple(np.add(np.multiply(left_shoulder, [image.shape[1], image.shape[0]]).astype(int), (-20, -20)))
            text_pos_shoulder_right = tuple(np.add(np.multiply(right_shoulder, [image.shape[1], image.shape[0]]).astype(int), (-20, -20)))

            draw_angles_with_background(image, right_angle, text_pos_right)
            draw_angles_with_background(image, left_angle, text_pos_left)
            draw_angles_with_background(image, left_shoulder_angle, text_pos_shoulder_left)
            draw_angles_with_background(image, right_shoulder_angle, text_pos_shoulder_right)

        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=5),
                                  mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2))

        cv2_imshow(image)
        frame_count += 1

cap.release()
cv2.destroyAllWindows()